In [1]:
def load_dataset():
    "Load the sample dataset."
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
    #return [['milk', 'butter', 'tea'], ['tea', 'ice', 'milk'], ['ice', 'orange', 'milk', 'butter'], ['milk', 'butter']]

In [2]:
from numpy import *

In [3]:
load_dataset()

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [1]:
def createC1(dataset):
    "Create a list of candidate item sets of size one."
    c1 = []
    for transaction in dataset:
        #print(transaction)
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    #frozenset because it will be a key of a dictionary.
    return list(map(frozenset, c1))


In [31]:
X =load_dataset()
createC1(X)2

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [32]:
def scanD(dataset, candidates, min_support):
    "Returns all candidates that meets a minimum support level"
    sscnt = {}
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1

    num_items = float(len(dataset))
    retlist = []
    support_data = {}
    for key in sscnt:
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support
    return retlist, support_data


In [33]:
dataset = load_dataset()

In [34]:
dataset

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [35]:
c1 = createC1(dataset)

In [36]:
c1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [37]:
D =  list(map(set,dataset))

In [38]:
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [39]:
L1,SD = scanD(D,c1,0.25)

In [40]:
L1

[frozenset({5}),
 frozenset({2}),
 frozenset({4}),
 frozenset({3}),
 frozenset({1})]

In [15]:
SD

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

In [30]:
x = frozenset(({5}), frozenset({2}), frozenset({3}), frozenset({1}))
y=frozenset(({5}), frozenset({2}))
(x | y)

TypeError: frozenset expected at most 1 arguments, got 4

In [17]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort() 
            L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
                #print(retList)
        
    return retList

In [18]:
L1, supportData = scanD(dataset, c1, 0.5)
L = [L1]
k = 2
print(L)
print(len(L[0]))
Ck = aprioriGen(L[k-2], k)
print(Ck)

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]]
4
[frozenset({2, 5}), frozenset({3, 5}), frozenset({1, 5}), frozenset({2, 3}), frozenset({1, 2}), frozenset({1, 3})]


In [19]:
def apriori(dataSet, minSupport = 0.40):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
        
       
    return L, supportData

In [20]:
L,SD = apriori(load_dataset())

In [21]:
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [22]:
SD

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

In [23]:
print(L[1])
Lk=L[1]
k=2
retList=[]
L1 = list(Lk[0])[:]

L2 = list(Lk[1])[:]

print('---',Lk[1] | Lk[3],'dfghjk')
L1.sort() 
L2.sort()
if L1==L2: #if first k-2 elements are equal
    retList.append(Lk[0] | Lk[1]) #set union
                #print(retList)
print(retList)
#aprioriGen(L[0],2)

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]
--- frozenset({1, 3, 5}) dfghjk
[]


In [24]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            print(freqSet)
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList   

In [25]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [26]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [27]:
L,suppData= apriori(dataset,minSupport=0.7)
print(L)
print(suppData)

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]
{frozenset({1}): 0.5, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({2}): 0.75, frozenset({5}): 0.75, frozenset({2, 5}): 0.75, frozenset({3, 5}): 0.5, frozenset({2, 3}): 0.5}


In [28]:
rules= generateRules(L,suppData, minConf=0.7)


frozenset({2, 5})
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
